In [11]:
import numpy as np
import pandas as pd

from scipy import stats

In [12]:
path_catalog = "../../Bases_de_datos/Data_catalog.csv"
iso = "../../Bases_de_datos/Country_ISO_code.csv"
path_v = "../data/variables.csv"
ix = "alpha-3"

df_c = pd.read_csv(path_catalog)
df_iso = pd.read_csv(iso).set_index(ix)
data_0 = pd.DataFrame(index = df_iso.index)
data_s   = data_0.copy()
data_c   = data_0.copy()
data_g   = data_0.copy()
data_q_s = data_0.copy()
data_q_c = data_0.copy()
data_q_g = data_0.copy()

df_v = pd.read_csv(path_v)

In [14]:
for cat in df_v["Category"].unique():
    data   = data_0.copy()
    data_q = data_0.copy()

    for row in df_v[ (df_v["Category"] == cat) ].itertuples():

        v = row.Variable
        if row.Origin == "World Bank": nr = 3
        else: nr = 0
        
        df = pd.read_csv( "../../"
            + df_c.loc[df_c["ID"]==row.ID, "Path"].iloc[0]
            + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
            skiprows = nr, low_memory = False )
        
        if row.Origin == "OECD":
            df = df[ df["MEASURE"] == v ]
            df = df.rename(columns = {"REF_AREA": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            
            df_i = df[ (df["MEASURE"]==v) ]
            
            if v == "POP_NETMIGR":
                df_i["OBS_VALUE"] = np.abs( df_i["OBS_VALUE"] )
            
            elif v in ["HD_POP_EXP", "HD_TN_POP_EXP", "ID_POP_EXP",
                "99PT_LAND_EXP", "99PT_CROP_EXP", "EP_LAND_EXP"]:
                df_i = df_i[ (df_i["DURATION"]==row.DURATION) ]
            elif v in ["TEMP_CHANGE", "ED_CROP_ANOM", "EP_CHANGE"]:
                df_i = df_i.set_index(ix)
                df_i["OBS_VALUE"] = df_i.reset_index()[
                    [ix, "OBS_VALUE"] ].groupby(ix).mean()
                df_i = df_i.reset_index()
            elif v == "UTCI_POP_EXP":
                df_i = df_i[ (df_i["HEAT_STRESS"]==row.HEAT_STRESS) ]
            elif v in ["CF_BUILT_EXP", "CF_POP_EXP", "CF_CROP_EXP",
                "RF_BUILT_EXP", "RF_POP_EXP", "RF_CROP_EXP"]:
                df_i = df_i[ (df_i["RET_PERIOD"]==row.RET_PERIOD) ]
            elif v == "BURN_LAND_AREA":
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            elif v in ["CY_BUILT_EXP", "CY_POP_EXP"]:
                df_i = df_i[
                    (df["HURRICANE_WIND_SCALE"]==row.HURRICANE_WIND_SCALE) ]
            elif v == "POP_EXP_POL":
                df_i = df_i[ (df_i["EXPOSURE_LEVEL"]==row.EXPOSURE_LEVEL) ]

            elif v == "ECRATE":
                df_i = df_i[ (df_i["SECTOR"]==row.SECTOR)
                & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
            elif v in ["GPAT_DE", "WATER_FW"]:
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            
            data[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]

        elif row.Origin == "Global Data Lab":
            df = df.rename(columns = {"ISO_Code": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            data[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
        
        elif row.Origin == "World Bank":
            df = df.rename(columns = {"Country Code": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            data[v] = df.set_index(ix).iloc[ :, 4:-1
                ].ffill(axis = 1).iloc[:, -1]
        
        elif row.Origin == "UNESCO UIS":
            if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
                df.columns = [ x.lower() for x in df.columns ]
            df = df.rename(columns = {"country_id": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            if row.PERCENTAGE == "Yes": pop = data["200101"]
            else: pop = 1
            data[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
                ).drop_duplicates(ix, keep="last"
                ).set_index(ix)["value"] / pop )
        
        # corregir random
        #data_q[v] = ( 1 + pd.qcut( data[v]
        #    + np.random.random(data[v].shape[0])/10000, 20, labels = False ) )
        # corregir random
        data_q[v] = stats.percentileofscore(
            data[v], data[v], nan_policy = "omit" ) / 10
        if row.Negative_indicator:
            data_q[v] = 10 - data_q[v]

    #data_q = data_q / 2
    data_q["Mean"] = data_q.mean(axis = 1)
    data_q["Count"] = data_q.count(axis = 1)
    data_q = data_q[ data_q["Count"] > 1 ].sort_values("Mean")

    if   cat == df_v["Category"].unique()[0]:
        data_q_s = data_q.copy()
        data_s   = data.copy()
        data_q_s.to_csv("../results/Deciles_Socioeconomico.csv", index = True)
        data_c.to_csv("../results/Valores_Socioeconomico.csv", index = True)
    elif cat == df_v["Category"].unique()[1]:
        data_q_c = data_q.copy()
        data_c   = data.copy()
        data_q_c.to_csv("../results/Deciles_Climatico.csv", index = True)
        data_c.to_csv("../results/Valores_Climatico.csv", index = True)
    elif cat == df_v["Category"].unique()[2]:
        data_q_g = data_q.copy()
        data_g   = data.copy()
        data_q_g.to_csv("../results/Deciles_Gobernanza.csv", index = True)
        data_g.to_csv("../results/Valores_Gobernanza.csv", index = True)

data_q_c

,HD_POP_EXP,HD_TN_POP_EXP,ID_POP_EXP,UTCI_POP_EXP,TEMP_CHANGE,CF_BUILT_EXP,CF_POP_EXP,CF_CROP_EXP,ED_CROP_ANOM,99PT_LAND_EXP,...,FT_FORE_EXP,CY_BUILT_EXP,CY_POP_EXP,W_BUILT_EXP,W_POP_EXP,MEAN_POP,POP_EXP_POL,EP_LAND_EXP,Mean,Count
alpha-3,,,,,,,,,,,,,,,,,,,,,
IND,0.619835,0.619835,1.528926,0.822511,6.963563,2.711864,5.721925,3.423913,0.324324,0.607287,...,2.631579,5.022624,5.0,6.493213,3.679654,0.742358,1.310044,2.955466,2.566768,26
EGY,0.578512,0.578512,5.805785,1.168831,2.429150,0.790960,1.497326,0.380435,0.864865,5.303644,...,0.445344,5.022624,5.0,6.493213,6.861472,0.436681,0.698690,7.995951,2.775634,26
CHN,2.561983,2.396694,0.578512,1.645022,4.251012,0.677966,1.336898,1.086957,7.135135,5.303644,...,4.331984,5.022624,5.0,2.986425,3.160173,1.572052,1.572052,4.048583,3.093866,26
IRQ,0.247934,0.289256,5.805785,0.086580,0.607287,2.937853,4.224599,3.858696,1.405405,5.303644,...,0.445344,5.022624,5.0,6.493213,3.722944,1.310044,1.266376,7.995951,3.107949,26
TCD,0.165289,0.165289,5.805785,0.476190,1.497976,NaN,NaN,NaN,8.972973,5.303644,...,0.445344,5.022624,5.0,6.493213,6.861472,0.829694,1.179039,5.344130,3.182983,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SXM,6.735537,6.714876,5.805785,NaN,9.170040,NaN,NaN,NaN,NaN,5.303644,...,7.995951,NaN,NaN,NaN,NaN,NaN,NaN,7.995951,7.340901,11
MAF,6.735537,6.714876,5.805785,NaN,9.170040,NaN,NaN,NaN,NaN,5.303644,...,7.995951,NaN,NaN,NaN,NaN,NaN,NaN,7.995951,7.340901,11
AIA,6.735537,6.714876,5.805785,6.666667,9.230769,8.418079,9.598930,8.695652,NaN,5.303644,...,7.995951,5.022624,5.0,6.493213,6.861472,8.646288,6.550218,7.995951,7.355601,25


In [15]:
res_names = [ "Socioeconomic resilience",
    "Climate resilience", "Governance resilience" ]

df_iso[ res_names[0]  ] = data_q_s.loc[
    data_q_s["Count"] > 11, "Mean" ]
df_iso[ res_names[1]  ] = data_q_c.loc[
    data_q_c["Count"] > 11, "Mean" ]
df_iso[ res_names[2]  ] = data_q_g.loc[
    data_q_g["Count"] > 3, "Mean" ]

df_nz = pd.read_csv( "../../" +
    df_c.loc[ df_c["Origin"]=="Climate Action Tracker", "Path" ].iloc[1] +
    df_c.loc[ df_c["Origin"]=="Climate Action Tracker", "Filename" ].iloc[1],
    usecols = ["ISO", "Overall Rating"] )
df_nz = df_nz.rename( columns = {"ISO": ix} ).set_index(ix)
df_iso["Net Zero Rating"] = df_nz["Overall Rating"]
df_iso["Net Zero Rating"] = df_iso["Net Zero Rating"].where(
    ~df_iso["Net Zero Rating"].isnull(), "No target" )

df_res = df_iso.drop( ["country-code", "iso_3166-2", "intermediate-region",
    "region-code", "sub-region-code", "intermediate-region-code"], axis = 1 )

df_res

df_rnz = df_res.loc[ df_res["Net Zero Rating"] != "No target",
    ["name", "Net Zero Rating", "alpha-2"] + res_names ]

df_rnz.to_csv( "../results/resumen_categorias.csv", index = True )

df_rnz

,name,Net Zero Rating,alpha-2,Socioeconomic resilience,Climate resilience,Governance resilience
alpha-3,,,,,,
ARG,Argentina,Poor,AR,5.355912,3.699203,5.357407
AUS,Australia,Poor,AU,6.173538,4.323282,5.009759
BTN,Bhutan,Information incomplete,BT,5.707487,4.988717,3.606019
BRA,Brazil,Information incomplete,BR,5.530113,4.236506,5.641233
CAN,Canada,Average,CA,5.882945,4.285114,4.982974
CHL,Chile,Acceptable,CL,5.672913,4.506615,5.887933
CHN,China,Poor,CN,4.689794,3.093866,4.745535
COL,Colombia,Acceptable,CO,4.391096,4.593764,5.883822
CRI,Costa Rica,Acceptable,CR,6.263940,5.403217,5.182518


In [16]:
import plotly.express as px
import plotly.graph_objects as go

df_g = df_rnz.copy()

n = list(range(df_rnz.shape[0]))
n.remove(0)
n.remove(2)
n.remove(4)
n.remove(5)
df_g = df_g.iloc[[2, 0, 4, 5] + n ]
for i in res_names:
    df_g[i] = df_rnz[i].apply(lambda x: float(f"{x:.2f}"))


fig = px.scatter( df_g, x = res_names[0], y = res_names[1],
    hover_name = "name", hover_data = res_names + ["Net Zero Rating"],
    size = res_names[2], color = "Net Zero Rating", opacity = 1, size_max = 25,
    color_discrete_sequence=["#808080", "#8B0000", "#DAA520", "#006400"]
)
#fig.update_traces(marker_color="rgba(0,0,0,0)")

for i, row in df_rnz.iterrows():
    country_iso = row["alpha-2"]
    fig.add_layout_image(
        dict(
            source=f"https://raw.githubusercontent.com/matahombres/CSS-Country-Flags-Rounded/master/flags/{country_iso}.png",
            xref="x", yref="y", xanchor="center", yanchor="middle",
            x=row[res_names[0]], y=row[res_names[1]],
            sizex=row[res_names[2]]/25, sizey=row[res_names[2]]/25,
            sizing="contain",
            #bordercolor = "green",
            opacity=1,
            layer="above"
        )
    )

fig.update_layout(height=600, width=1000, plot_bgcolor="#dfdfdf")

fig.show()